In [1]:
import lzma
import json
import pandas as pd
import numpy as np
import re
import glob
import os
import sys
import multiprocessing as mp
import pickle

In [3]:
os.chdir(r"C:\Users\erica\OneDrive - University of Southern California\Projects_at_USC\CKIDS_current\data")

In [4]:
os.getcwd()

'C:\\Users\\erica\\OneDrive - University of Southern California\\Projects_at_USC\\CKIDS_current\\data'

In [5]:
# lists of file names
# json xz
json_files = glob.glob("*_UTC.json")
# descrip txts
txt_files = glob.glob("*_UTC.txt")
# location txts
loc_files = glob.glob("*_location.txt")
# comments jsons
cmt_files = glob.glob("*_comments.json")
# num of jpgs
jpg_files = glob.glob("*.jpg")



print(len(json_files))
print(len(txt_files))
print(len(loc_files))
print(len(cmt_files))
print(len(jpg_files))

265
256
168
194
357


In [6]:
# Add all json files to a dataframe
temp = pd.DataFrame()

path_to_json = r'C:\Users\erica\OneDrive - University of Southern California\Projects_at_USC\CKIDS_current\data' 

json_pattern = os.path.join(path_to_json,'*_UTC.json')
file_list = glob.glob(json_pattern)

for file in json_files:
    # print(file)
    data = pd.read_json(file, orient=str)
    postnode = data['node']
    temp = temp.append(postnode, ignore_index = True)

In [7]:
temp.shape

(265, 53)

In [8]:
temp.head(3)

,__typename,accessibility_caption,caption_is_edited,commenting_disabled_for_viewer,comments_disabled,dimensions,display_resources,display_url,edge_media_preview_comment,edge_media_preview_like,...,media_overlay_info,dash_info,encoding_status,is_published,product_type,title,video_duration,video_url,video_view_count,felix_profile_grid_crop
0,GraphImage,"Photo by urbanplates on January 31, 2020. Imag...",1.0,0.0,0.0,"{'height': 1080, 'width': 1080}","[{'config_height': 640, 'config_width': 640, '...",https://scontent-lax3-1.cdninstagram.com/v/t51...,"{'count': 4, 'edges': [{'node': {'created_at':...","{'count': 249, 'edges': []}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GraphSidecar,NaN,0.0,0.0,0.0,"{'height': 1080, 'width': 1080}","[{'config_height': 640, 'config_width': 640, '...",https://scontent-lax3-1.cdninstagram.com/v/t51...,"{'count': 2, 'edges': [{'node': {'created_at':...","{'count': 14, 'edges': []}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GraphSidecar,NaN,0.0,0.0,0.0,"{'height': 1080, 'width': 1080}","[{'config_height': 640, 'config_width': 640, '...",https://scontent-lax3-1.cdninstagram.com/v/t51...,"{'count': 0, 'edges': []}","{'count': 12, 'edges': []}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
json_fnames = [f.split('.json')[0] for f in json_files]
json_fnamesS = pd.Series(json_fnames)
temp['file_name'] = json_fnamesS

In [10]:
temp2 = temp[temp['__typename'] == "GraphImage"]
print(temp2.shape)

(218, 54)


In [12]:
# temp2.columns

In [13]:
# temp2.iloc[0]

In [11]:
df = temp2[['file_name', 'accessibility_caption', 'display_url', 'edge_liked_by', 'id', 'is_ad', 'location', 'owner', 'taken_at_timestamp']]

In [12]:
df.head()

,file_name,accessibility_caption,display_url,edge_liked_by,id,is_ad,location,owner,taken_at_timestamp
0,2020-02-01_00-30-09_UTC,"Photo by urbanplates on January 31, 2020. Imag...",https://scontent-lax3-1.cdninstagram.com/v/t51...,NaN,2233861020543364147,0.0,None,{'biography': 'Everyone deserves to eat this g...,1.580517e+09
3,2020-02-01_18-30-28_UTC,Photo by #dineLA in HiHo Cheeseburger with @hi...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 2235},2234404663954808455,0.0,"{'has_public_page': True, 'id': 28208364046247...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580582e+09
7,2020-02-02_15-10-30_UTC,Photo by @yourhoiness in HiHo Cheeseburger. Im...,https://scontent-lax3-2.cdninstagram.com/v/t51...,{'count': 4},2235028910851352065,0.0,"{'has_public_page': True, 'id': 28208364046247...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580656e+09
8,2020-02-02_16-00-16_UTC,"Photo by The Henry on February 02, 2020. Image...",https://scontent-lax3-1.cdninstagram.com/v/t51...,NaN,2235053941526362311,0.0,None,{'biography': '#FollowTheArrow to the greatest...,1.580659e+09
10,2020-02-03_00-05-07_UTC,Photo by HiHo 🍔 in HiHo Cheeseburger with @hih...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,NaN,2235297969438836147,0.0,"{'has_public_page': True, 'id': 13259326375585...",{'biography': 'Santa Monica • Mid-Wilshire The...,1.580688e+09


In [13]:
df.to_csv("post_jsons_df_fs.csv")

In [40]:
# Read in text descrip files (hashtags)

In [14]:
def read_txt(file):
    text = open(file, "r", encoding="utf8").read()
    return text   # return as string


def get_ht(text):
    ht = re.findall(r"#(\w+)", text)
    return ht

In [15]:
texts = []

for f in txt_files:
    text = open(f, 'r', encoding="utf8").read()
    texts.append(text) 

In [16]:
texts = list(map(read_txt, txt_files))
hashtags = list(map(get_ht, texts))

In [17]:
# get file names
txt_fnames = [f.split('.txt')[0] for f in txt_files]

# convert all to series, and concat them to a df
textsS = pd.Series(texts)
hashtagsS = pd.Series(hashtags)
txt_fnamesS = pd.Series(txt_fnames)

df_txt = pd.DataFrame(pd.concat([txt_fnamesS, textsS, hashtagsS], axis=1))
df_txt.columns = ['file_name', 'descrip', 'hashtags']

In [18]:
df_txt.to_csv("df_txt_fs.csv")

In [ ]:
# Join the json df and txt df by their common filename

In [19]:
print(df_txt.columns)
print(df.columns)

Index(['file_name', 'descrip', 'hashtags'], dtype='object')
Index(['file_name', 'accessibility_caption', 'display_url', 'edge_liked_by',
       'id', 'is_ad', 'location', 'owner', 'taken_at_timestamp'],
      dtype='object')


In [20]:
df_txt['file_name'].head()

0    2020-02-01_00-30-09_UTC
1    2020-02-01_10-23-03_UTC
2    2020-02-01_16-36-56_UTC
3    2020-02-01_18-30-28_UTC
4    2020-02-01_19-30-47_UTC
Name: file_name, dtype: object

In [21]:
df['file_name'].head()

0     2020-02-01_00-30-09_UTC
3     2020-02-01_18-30-28_UTC
7     2020-02-02_15-10-30_UTC
8     2020-02-02_16-00-16_UTC
10    2020-02-03_00-05-07_UTC
Name: file_name, dtype: object

In [22]:
# merge the post metadata (jsons) df and text descrip df (txt)
merged = pd.merge(df, df_txt, on='file_name')
print(merged.shape)

(208, 11)


In [23]:
merged.head()

,file_name,accessibility_caption,display_url,edge_liked_by,id,is_ad,location,owner,taken_at_timestamp,descrip,hashtags
0,2020-02-01_00-30-09_UTC,"Photo by urbanplates on January 31, 2020. Imag...",https://scontent-lax3-1.cdninstagram.com/v/t51...,NaN,2233861020543364147,0.0,None,{'biography': 'Everyone deserves to eat this g...,1.580517e+09,Staying cozy by the fireplace at our La Jolla ...,[]
1,2020-02-01_18-30-28_UTC,Photo by #dineLA in HiHo Cheeseburger with @hi...,https://scontent-lax3-1.cdninstagram.com/v/t51...,{'count': 2235},2234404663954808455,0.0,"{'has_public_page': True, 'id': 28208364046247...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580582e+09,Thank you for dining out during #dineLA Restau...,[dineLA]
2,2020-02-02_15-10-30_UTC,Photo by @yourhoiness in HiHo Cheeseburger. Im...,https://scontent-lax3-2.cdninstagram.com/v/t51...,{'count': 4},2235028910851352065,0.0,"{'has_public_page': True, 'id': 28208364046247...","{'blocked_by_viewer': False, 'edge_owner_to_ti...",1.580656e+09,Is this a sign?\n,[]
3,2020-02-02_16-00-16_UTC,"Photo by The Henry on February 02, 2020. Image...",https://scontent-lax3-1.cdninstagram.com/v/t51...,NaN,2235053941526362311,0.0,None,{'biography': '#FollowTheArrow to the greatest...,1.580659e+09,Sunday Brunch for the win! Sweeten up the last...,[]
4,2020-02-03_00-05-07_UTC,Photo by HiHo 🍔 in HiHo Cheeseburger with @hih...,https://scontent-sjc3-1.cdninstagram.com/v/t51...,NaN,2235297969438836147,0.0,"{'has_public_page': True, 'id': 13259326375585...",{'biography': 'Santa Monica • Mid-Wilshire The...,1.580688e+09,Game day essentials! 🏈 🍔#SuperBowlLIV 📷 @plant...,"[SuperBowlLIV, HiHoCheeseburger]"


In [24]:
merged.to_csv("posts_merged_df_fs.csv")